### Creat the new database "se4g"

In [4]:
import os
from dotenv import load_dotenv
import psycopg2

# Get info from .env file
load_dotenv()
db_name = os.getenv('DB_NAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
if not all([db_name, user, password, host]):
    raise ValueError("Plz set environment variables in .env file")

# Connect to the default database
conn = psycopg2.connect(dbname="postgres", user=user, password=password, host=host)
conn.autocommit = True
cursor = conn.cursor()

# Creat database
try:
    cursor.execute(f"CREATE DATABASE {db_name};")
except Exception as error:
    print(f"Error: {error}")
cursor.close()
conn.close()

### Creat PostGIS extension

In [5]:
# Creat extension
# Connect to the new database
conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
cursor = conn.cursor()

try:
    create_extension_query = """
    CREATE EXTENSION IF NOT EXISTS postgis;
    """
    cursor.execute(create_extension_query)
    conn.commit()
except Exception as error:
    print(f"Error: {error}")
cursor.close()    
conn.close()

### Insert Idro_GEO API region data

In [4]:
import requests

IdroGEOAPI = os.getenv("IDROGEO_API")
regionAPI =  IdroGEOAPI + "/regioni"
response = requests.get(regionAPI)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    
try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    cursor = conn.cursor()
    #Creat table
    creat_query = """
    CREATE TABLE IF NOT EXISTS regioni (
        uid INTEGER PRIMARY KEY,
        nome VARCHAR(100),
        osmid INTEGER
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    #Insert data
    insert_query = """
    INSERT INTO regioni (uid, nome, osmid)
    VALUES (%s, %s, %s);
    """
    for item in data:
        cursor.execute(insert_query, (item['uid'], item['nome'], item['osmid']))

    conn.commit()
    cursor.close()
    conn.close()
except Exception as error:
    print(f"Error: {error}")

### Insert region PIR data(flood&landslide)

In [5]:
try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    cursor = conn.cursor()
    # Flood table
    creat_query = """
    CREATE TABLE IF NOT EXISTS regioni_flood (
    uid INTEGER PRIMARY KEY,
    osmid INTEGER,
    nome VARCHAR(100),
    ar_kmq NUMERIC,
    ar_id_p3 NUMERIC,
    ar_id_p2 NUMERIC,
    ar_id_p1 NUMERIC,
    aridp3_p NUMERIC,
    aridp2_p NUMERIC,
    aridp1_p NUMERIC,
    pop_res011 INTEGER,
    pop_gio INTEGER,
    pop_gio_p NUMERIC,
    pop_adu INTEGER,
    pop_adu_p NUMERIC,
    pop_anz INTEGER,
    pop_anz_p NUMERIC,
    pop_idr_p3 INTEGER,
    pop_idr_p2 INTEGER,
    pop_idr_p1 INTEGER,
    popidp3_p NUMERIC,
    popidp2_p NUMERIC,
    popidp1_p NUMERIC,
    fam_tot INTEGER,
    fam_idr_p3 INTEGER,
    fam_idr_p2 INTEGER,
    fam_idr_p1 INTEGER,
    famidp3_p NUMERIC,
    famidp2_p NUMERIC,
    famidp1_p NUMERIC,
    ed_tot INTEGER,
    ed_idr_p3 INTEGER,
    ed_idr_p2 INTEGER,
    ed_idr_p1 INTEGER,
    edidp3_p NUMERIC,
    edidp2_p NUMERIC,
    edidp1_p NUMERIC,
    im_tot INTEGER,
    im_idr_p3 INTEGER,
    im_idr_p2 INTEGER,
    im_idr_p1 INTEGER,
    imidp3_p NUMERIC,
    imidp2_p NUMERIC,
    imidp1_p NUMERIC,
    n_vir INTEGER,
    bbcc_id_p3 INTEGER,
    bbcc_id_p2 INTEGER,
    bbcc_id_p1 INTEGER,
    bbccidp3_p NUMERIC,
    bbccidp2_p NUMERIC,
    bbccidp1_p NUMERIC
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    # Landslide table
    creat_query = """
    CREATE TABLE IF NOT EXISTS regioni_ls (
    uid INTEGER PRIMARY KEY,
    osmid INTEGER,
    nome VARCHAR(100),
    ar_kmq NUMERIC,
    ar_fr_p4 NUMERIC,
    ar_fr_p3 NUMERIC,
    ar_fr_p2 NUMERIC,
    ar_fr_p1 NUMERIC,
    ar_fr_aa NUMERIC,
    ar_fr_p3p4 NUMERIC,
    ar_frp4_p NUMERIC,
    ar_frp3_p NUMERIC,
    ar_frp2_p NUMERIC,
    ar_frp1_p NUMERIC,
    ar_fraa_p NUMERIC,
    ar_frp3p4p NUMERIC,
    pop_fr_p4 NUMERIC,
    pop_fr_p3 NUMERIC,
    pop_fr_p2 NUMERIC,
    pop_fr_p1 NUMERIC,
    pop_fr_aa NUMERIC,
    popfr_p3p4 NUMERIC,
    popfrp4_p NUMERIC,
    popfrp3_p NUMERIC,
    popfrp2_p NUMERIC,
    popfrp1_p NUMERIC,
    popfraa_p NUMERIC,
    popfrp3p4p NUMERIC,
    fam_fr_p4 NUMERIC,
    fam_fr_p3 NUMERIC,
    fam_fr_p2 NUMERIC,
    fam_fr_p1 NUMERIC,
    fam_fr_aa NUMERIC,
    famfr_p3p4 NUMERIC,
    famfrp4_p NUMERIC,
    famfrp3_p NUMERIC,
    famfrp2_p NUMERIC,
    famfrp1_p NUMERIC,
    famfraa_p NUMERIC,
    famfrp3p4p NUMERIC,
    ed_fr_p4 NUMERIC,
    ed_fr_p3 NUMERIC,
    ed_fr_p2 NUMERIC,
    ed_fr_p1 NUMERIC,
    ed_fr_aa NUMERIC,
    ed_fr_p3p4 NUMERIC,
    edfrp4_p NUMERIC,
    edfrp3_p NUMERIC,
    edfrp2_p NUMERIC,
    edfrp1_p NUMERIC,
    edfraa_p NUMERIC,
    edfrp3p4p NUMERIC,
    im_fr_p4 NUMERIC,
    im_fr_p3 NUMERIC,
    im_fr_p2 NUMERIC,
    im_fr_p1 NUMERIC,
    im_fr_aa NUMERIC,
    imfr_p3p4 NUMERIC,
    imfrp4_p NUMERIC,
    imfrp3_p NUMERIC,
    imfrp2_p NUMERIC,
    imfrp1_p NUMERIC,
    imfraa_p NUMERIC,
    imfrp3p4p NUMERIC,
    bbcc_fr_p4 NUMERIC,
    bbcc_fr_p3 NUMERIC,
    bbcc_fr_p2 NUMERIC,
    bbcc_fr_p1 NUMERIC,
    bbcc_fr_aa NUMERIC,
    bbccfrp3p4 NUMERIC,
    bbccfrp4_p NUMERIC,
    bbccfrp3_p NUMERIC,
    bbccfrp2_p NUMERIC,
    bbccfrp1_p NUMERIC,
    bbccfraa_p NUMERIC,
    bbccfrp34p NUMERIC
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    # Search table
    search_query = "SELECT uid, nome FROM regioni"
    cursor.execute(search_query)
    results = cursor.fetchall()
    for i in results:
        uid = i[0]
        nome = i[1]
        regionIdAPI =  regionAPI + "/" + str(uid)
        response = requests.get(regionIdAPI)
        if response.status_code == 200:
            res = response.json()
            if nome == res['nome']:
                # Insert flood data
                insert_query = """
                INSERT INTO regioni_flood(
                    uid,osmid, nome, ar_kmq, ar_id_p3, ar_id_p2, ar_id_p1,
                    aridp3_p, aridp2_p, aridp1_p, pop_res011, pop_gio,
                    pop_gio_p, pop_adu, pop_adu_p, pop_anz, pop_anz_p,
                    pop_idr_p3, pop_idr_p2, pop_idr_p1, popidp3_p, popidp2_p,
                    popidp1_p, fam_tot, fam_idr_p3, fam_idr_p2, fam_idr_p1,
                    famidp3_p, famidp2_p, famidp1_p, ed_tot, ed_idr_p3,
                    ed_idr_p2, ed_idr_p1, edidp3_p, edidp2_p, edidp1_p,
                    im_tot, im_idr_p3, im_idr_p2, im_idr_p1, imidp3_p,
                    imidp2_p, imidp1_p, n_vir, bbcc_id_p3, bbcc_id_p2,
                    bbcc_id_p1, bbccidp3_p, bbccidp2_p, bbccidp1_p
                )
                VALUES(
                    %(uid)s, %(osmid)s, %(nome)s, %(ar_kmq)s, %(ar_id_p3)s, %(ar_id_p2)s, %(ar_id_p1)s,
                    %(aridp3_p)s, %(aridp2_p)s, %(aridp1_p)s, %(pop_res011)s, %(pop_gio)s,
                    %(pop_gio_p)s, %(pop_adu)s, %(pop_adu_p)s, %(pop_anz)s, %(pop_anz_p)s,
                    %(pop_idr_p3)s, %(pop_idr_p2)s, %(pop_idr_p1)s, %(popidp3_p)s, %(popidp2_p)s,
                    %(popidp1_p)s, %(fam_tot)s, %(fam_idr_p3)s, %(fam_idr_p2)s, %(fam_idr_p1)s,
                    %(famidp3_p)s, %(famidp2_p)s, %(famidp1_p)s, %(ed_tot)s, %(ed_idr_p3)s,
                    %(ed_idr_p2)s, %(ed_idr_p1)s, %(edidp3_p)s, %(edidp2_p)s, %(edidp1_p)s,
                    %(im_tot)s, %(im_idr_p3)s, %(im_idr_p2)s, %(im_idr_p1)s, %(imidp3_p)s,
                    %(imidp2_p)s, %(imidp1_p)s, %(n_vir)s, %(bbcc_id_p3)s, %(bbcc_id_p2)s,
                    %(bbcc_id_p1)s, %(bbccidp3_p)s, %(bbccidp2_p)s, %(bbccidp1_p)s
                )
                """
                cursor.execute(insert_query, res)
                conn.commit()
                # Insert landslide data
                insert_query = """
                INSERT INTO regioni_ls(
                    uid, osmid, nome, ar_kmq, ar_fr_p4, ar_fr_p3, ar_fr_p2, ar_fr_p1, ar_fr_aa,
                    ar_fr_p3p4, ar_frp4_p, ar_frp3_p, ar_frp2_p, ar_frp1_p, ar_fraa_p,
                    ar_frp3p4p, pop_fr_p4, pop_fr_p3, pop_fr_p2, pop_fr_p1, pop_fr_aa,
                    popfr_p3p4, popfrp4_p, popfrp3_p, popfrp2_p, popfrp1_p, popfraa_p,
                    popfrp3p4p, fam_fr_p4, fam_fr_p3, fam_fr_p2, fam_fr_p1, fam_fr_aa,
                    famfr_p3p4, famfrp4_p, famfrp3_p, famfrp2_p, famfrp1_p, famfraa_p,
                    famfrp3p4p, ed_fr_p4, ed_fr_p3, ed_fr_p2, ed_fr_p1, ed_fr_aa,
                    ed_fr_p3p4, edfrp4_p, edfrp3_p, edfrp2_p, edfrp1_p, edfraa_p,
                    edfrp3p4p, im_fr_p4, im_fr_p3, im_fr_p2, im_fr_p1, im_fr_aa,
                    imfr_p3p4, imfrp4_p, imfrp3_p, imfrp2_p, imfrp1_p, imfraa_p,
                    imfrp3p4p, bbcc_fr_p4, bbcc_fr_p3, bbcc_fr_p2, bbcc_fr_p1, bbcc_fr_aa,
                    bbccfrp3p4, bbccfrp4_p, bbccfrp3_p, bbccfrp2_p, bbccfrp1_p, bbccfraa_p, bbccfrp34p
                )
                VALUES(
                    %(uid)s, %(osmid)s, %(nome)s, %(ar_kmq)s, %(ar_fr_p4)s, %(ar_fr_p3)s, %(ar_fr_p2)s, %(ar_fr_p1)s, %(ar_fr_aa)s,%(ar_fr_p3p4)s, %(ar_frp4_p)s, %(ar_frp3_p)s, %(ar_frp2_p)s, %(ar_frp1_p)s, %(ar_fraa_p)s,
                    %(ar_frp3p4p)s, %(pop_fr_p4)s, %(pop_fr_p3)s, %(pop_fr_p2)s, %(pop_fr_p1)s, %(pop_fr_aa)s,
                    %(popfr_p3p4)s, %(popfrp4_p)s, %(popfrp3_p)s, %(popfrp2_p)s, %(popfrp1_p)s, %(popfraa_p)s,
                    %(popfrp3p4p)s, %(fam_fr_p4)s, %(fam_fr_p3)s, %(fam_fr_p2)s, %(fam_fr_p1)s, %(fam_fr_aa)s,
                    %(famfr_p3p4)s, %(famfrp4_p)s, %(famfrp3_p)s, %(famfrp2_p)s, %(famfrp1_p)s, %(famfraa_p)s,
                    %(famfrp3p4p)s, %(ed_fr_p4)s, %(ed_fr_p3)s, %(ed_fr_p2)s, %(ed_fr_p1)s, %(ed_fr_aa)s,
                    %(ed_fr_p3p4)s, %(edfrp4_p)s, %(edfrp3_p)s, %(edfrp2_p)s, %(edfrp1_p)s, %(edfraa_p)s,
                    %(edfrp3p4p)s, %(im_fr_p4)s, %(im_fr_p3)s, %(im_fr_p2)s, %(im_fr_p1)s, %(im_fr_aa)s,
                    %(imfr_p3p4)s, %(imfrp4_p)s, %(imfrp3_p)s, %(imfrp2_p)s, %(imfrp1_p)s, %(imfraa_p)s,
                    %(imfrp3p4p)s, %(bbcc_fr_p4)s, %(bbcc_fr_p3)s, %(bbcc_fr_p2)s, %(bbcc_fr_p1)s, %(bbcc_fr_aa)s,
                    %(bbccfrp3p4)s, %(bbccfrp4_p)s, %(bbccfrp3_p)s, %(bbccfrp2_p)s, %(bbccfrp1_p)s, %(bbccfraa_p)s,
                    %(bbccfrp34p)s
                )
                """
                cursor.execute(insert_query, res)
                conn.commit()
        else:
            raise response.status_code
    cursor.close()
    conn.close()
except Exception as error:
    print(f"Error: {error}")

### Insert Idro_GEO API province data

In [6]:
provinceAPI =  IdroGEOAPI + "/province"
response = requests.get(provinceAPI)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    
try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    cursor = conn.cursor()
    #Creat table
    creat_query = """
    CREATE TABLE IF NOT EXISTS province (
        uid INTEGER PRIMARY KEY,
        nome VARCHAR(100),
        osmid INTEGER,
        cod_reg INTEGER
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    #Insert data
    insert_query = """
    INSERT INTO province (uid, nome, osmid, cod_reg)
    VALUES (%s, %s, %s, %s);
    """
    for item in data:
        cursor.execute(insert_query, (item['uid'], item['nome'], item['osmid'], item['cod_reg']))

    conn.commit()
    cursor.close()
    conn.close()
except Exception as error:
    print(f"Error: {error}")

### Insert province PIR data(flood&landslide)

In [7]:
try:
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    cursor = conn.cursor()
    # Flood table
    creat_query = """
    CREATE TABLE IF NOT EXISTS province_flood (
    uid INTEGER PRIMARY KEY,
    osmid INTEGER,
    nome VARCHAR(100),
    ar_kmq NUMERIC,
    ar_id_p3 NUMERIC,
    ar_id_p2 NUMERIC,
    ar_id_p1 NUMERIC,
    aridp3_p NUMERIC,
    aridp2_p NUMERIC,
    aridp1_p NUMERIC,
    pop_res011 INTEGER,
    pop_gio INTEGER,
    pop_gio_p NUMERIC,
    pop_adu INTEGER,
    pop_adu_p NUMERIC,
    pop_anz INTEGER,
    pop_anz_p NUMERIC,
    pop_idr_p3 INTEGER,
    pop_idr_p2 INTEGER,
    pop_idr_p1 INTEGER,
    popidp3_p NUMERIC,
    popidp2_p NUMERIC,
    popidp1_p NUMERIC,
    fam_tot INTEGER,
    fam_idr_p3 INTEGER,
    fam_idr_p2 INTEGER,
    fam_idr_p1 INTEGER,
    famidp3_p NUMERIC,
    famidp2_p NUMERIC,
    famidp1_p NUMERIC,
    ed_tot INTEGER,
    ed_idr_p3 INTEGER,
    ed_idr_p2 INTEGER,
    ed_idr_p1 INTEGER,
    edidp3_p NUMERIC,
    edidp2_p NUMERIC,
    edidp1_p NUMERIC,
    im_tot INTEGER,
    im_idr_p3 INTEGER,
    im_idr_p2 INTEGER,
    im_idr_p1 INTEGER,
    imidp3_p NUMERIC,
    imidp2_p NUMERIC,
    imidp1_p NUMERIC,
    n_vir INTEGER,
    bbcc_id_p3 INTEGER,
    bbcc_id_p2 INTEGER,
    bbcc_id_p1 INTEGER,
    bbccidp3_p NUMERIC,
    bbccidp2_p NUMERIC,
    bbccidp1_p NUMERIC
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    # Landslide table
    creat_query = """
    CREATE TABLE IF NOT EXISTS province_ls (
    uid INTEGER PRIMARY KEY,
    osmid INTEGER,
    nome VARCHAR(100),
    ar_kmq NUMERIC,
    ar_fr_p4 NUMERIC,
    ar_fr_p3 NUMERIC,
    ar_fr_p2 NUMERIC,
    ar_fr_p1 NUMERIC,
    ar_fr_aa NUMERIC,
    ar_fr_p3p4 NUMERIC,
    ar_frp4_p NUMERIC,
    ar_frp3_p NUMERIC,
    ar_frp2_p NUMERIC,
    ar_frp1_p NUMERIC,
    ar_fraa_p NUMERIC,
    ar_frp3p4p NUMERIC,
    pop_fr_p4 NUMERIC,
    pop_fr_p3 NUMERIC,
    pop_fr_p2 NUMERIC,
    pop_fr_p1 NUMERIC,
    pop_fr_aa NUMERIC,
    popfr_p3p4 NUMERIC,
    popfrp4_p NUMERIC,
    popfrp3_p NUMERIC,
    popfrp2_p NUMERIC,
    popfrp1_p NUMERIC,
    popfraa_p NUMERIC,
    popfrp3p4p NUMERIC,
    fam_fr_p4 NUMERIC,
    fam_fr_p3 NUMERIC,
    fam_fr_p2 NUMERIC,
    fam_fr_p1 NUMERIC,
    fam_fr_aa NUMERIC,
    famfr_p3p4 NUMERIC,
    famfrp4_p NUMERIC,
    famfrp3_p NUMERIC,
    famfrp2_p NUMERIC,
    famfrp1_p NUMERIC,
    famfraa_p NUMERIC,
    famfrp3p4p NUMERIC,
    ed_fr_p4 NUMERIC,
    ed_fr_p3 NUMERIC,
    ed_fr_p2 NUMERIC,
    ed_fr_p1 NUMERIC,
    ed_fr_aa NUMERIC,
    ed_fr_p3p4 NUMERIC,
    edfrp4_p NUMERIC,
    edfrp3_p NUMERIC,
    edfrp2_p NUMERIC,
    edfrp1_p NUMERIC,
    edfraa_p NUMERIC,
    edfrp3p4p NUMERIC,
    im_fr_p4 NUMERIC,
    im_fr_p3 NUMERIC,
    im_fr_p2 NUMERIC,
    im_fr_p1 NUMERIC,
    im_fr_aa NUMERIC,
    imfr_p3p4 NUMERIC,
    imfrp4_p NUMERIC,
    imfrp3_p NUMERIC,
    imfrp2_p NUMERIC,
    imfrp1_p NUMERIC,
    imfraa_p NUMERIC,
    imfrp3p4p NUMERIC,
    bbcc_fr_p4 NUMERIC,
    bbcc_fr_p3 NUMERIC,
    bbcc_fr_p2 NUMERIC,
    bbcc_fr_p1 NUMERIC,
    bbcc_fr_aa NUMERIC,
    bbccfrp3p4 NUMERIC,
    bbccfrp4_p NUMERIC,
    bbccfrp3_p NUMERIC,
    bbccfrp2_p NUMERIC,
    bbccfrp1_p NUMERIC,
    bbccfraa_p NUMERIC,
    bbccfrp34p NUMERIC
    );
    """
    cursor.execute(creat_query)
    conn.commit()
    # Search table
    search_query = "SELECT uid, nome FROM province"
    cursor.execute(search_query)
    results = cursor.fetchall()
    for i in results:
        uid = i[0]
        nome = i[1]
        provinceIdAPI =  provinceAPI + "/" + str(uid)
        response = requests.get(provinceIdAPI)
        if response.status_code == 200:
            res = response.json()
            if nome == res['nome']:
                # Insert flood data
                insert_query = """
                INSERT INTO province_flood(
                    uid,osmid, nome, ar_kmq, ar_id_p3, ar_id_p2, ar_id_p1,
                    aridp3_p, aridp2_p, aridp1_p, pop_res011, pop_gio,
                    pop_gio_p, pop_adu, pop_adu_p, pop_anz, pop_anz_p,
                    pop_idr_p3, pop_idr_p2, pop_idr_p1, popidp3_p, popidp2_p,
                    popidp1_p, fam_tot, fam_idr_p3, fam_idr_p2, fam_idr_p1,
                    famidp3_p, famidp2_p, famidp1_p, ed_tot, ed_idr_p3,
                    ed_idr_p2, ed_idr_p1, edidp3_p, edidp2_p, edidp1_p,
                    im_tot, im_idr_p3, im_idr_p2, im_idr_p1, imidp3_p,
                    imidp2_p, imidp1_p, n_vir, bbcc_id_p3, bbcc_id_p2,
                    bbcc_id_p1, bbccidp3_p, bbccidp2_p, bbccidp1_p
                )
                VALUES(
                    %(uid)s, %(osmid)s, %(nome)s, %(ar_kmq)s, %(ar_id_p3)s, %(ar_id_p2)s, %(ar_id_p1)s,
                    %(aridp3_p)s, %(aridp2_p)s, %(aridp1_p)s, %(pop_res011)s, %(pop_gio)s,
                    %(pop_gio_p)s, %(pop_adu)s, %(pop_adu_p)s, %(pop_anz)s, %(pop_anz_p)s,
                    %(pop_idr_p3)s, %(pop_idr_p2)s, %(pop_idr_p1)s, %(popidp3_p)s, %(popidp2_p)s,
                    %(popidp1_p)s, %(fam_tot)s, %(fam_idr_p3)s, %(fam_idr_p2)s, %(fam_idr_p1)s,
                    %(famidp3_p)s, %(famidp2_p)s, %(famidp1_p)s, %(ed_tot)s, %(ed_idr_p3)s,
                    %(ed_idr_p2)s, %(ed_idr_p1)s, %(edidp3_p)s, %(edidp2_p)s, %(edidp1_p)s,
                    %(im_tot)s, %(im_idr_p3)s, %(im_idr_p2)s, %(im_idr_p1)s, %(imidp3_p)s,
                    %(imidp2_p)s, %(imidp1_p)s, %(n_vir)s, %(bbcc_id_p3)s, %(bbcc_id_p2)s,
                    %(bbcc_id_p1)s, %(bbccidp3_p)s, %(bbccidp2_p)s, %(bbccidp1_p)s
                )
                """
                cursor.execute(insert_query, res)
                conn.commit()
                # Insert landslide data
                insert_query = """
                INSERT INTO province_ls(
                    uid, osmid, nome, ar_kmq, ar_fr_p4, ar_fr_p3, ar_fr_p2, ar_fr_p1, ar_fr_aa,
                    ar_fr_p3p4, ar_frp4_p, ar_frp3_p, ar_frp2_p, ar_frp1_p, ar_fraa_p,
                    ar_frp3p4p, pop_fr_p4, pop_fr_p3, pop_fr_p2, pop_fr_p1, pop_fr_aa,
                    popfr_p3p4, popfrp4_p, popfrp3_p, popfrp2_p, popfrp1_p, popfraa_p,
                    popfrp3p4p, fam_fr_p4, fam_fr_p3, fam_fr_p2, fam_fr_p1, fam_fr_aa,
                    famfr_p3p4, famfrp4_p, famfrp3_p, famfrp2_p, famfrp1_p, famfraa_p,
                    famfrp3p4p, ed_fr_p4, ed_fr_p3, ed_fr_p2, ed_fr_p1, ed_fr_aa,
                    ed_fr_p3p4, edfrp4_p, edfrp3_p, edfrp2_p, edfrp1_p, edfraa_p,
                    edfrp3p4p, im_fr_p4, im_fr_p3, im_fr_p2, im_fr_p1, im_fr_aa,
                    imfr_p3p4, imfrp4_p, imfrp3_p, imfrp2_p, imfrp1_p, imfraa_p,
                    imfrp3p4p, bbcc_fr_p4, bbcc_fr_p3, bbcc_fr_p2, bbcc_fr_p1, bbcc_fr_aa,
                    bbccfrp3p4, bbccfrp4_p, bbccfrp3_p, bbccfrp2_p, bbccfrp1_p, bbccfraa_p, bbccfrp34p
                )
                VALUES(
                    %(uid)s, %(osmid)s, %(nome)s, %(ar_kmq)s, %(ar_fr_p4)s, %(ar_fr_p3)s, %(ar_fr_p2)s, %(ar_fr_p1)s, %(ar_fr_aa)s,%(ar_fr_p3p4)s, %(ar_frp4_p)s, %(ar_frp3_p)s, %(ar_frp2_p)s, %(ar_frp1_p)s, %(ar_fraa_p)s,
                    %(ar_frp3p4p)s, %(pop_fr_p4)s, %(pop_fr_p3)s, %(pop_fr_p2)s, %(pop_fr_p1)s, %(pop_fr_aa)s,
                    %(popfr_p3p4)s, %(popfrp4_p)s, %(popfrp3_p)s, %(popfrp2_p)s, %(popfrp1_p)s, %(popfraa_p)s,
                    %(popfrp3p4p)s, %(fam_fr_p4)s, %(fam_fr_p3)s, %(fam_fr_p2)s, %(fam_fr_p1)s, %(fam_fr_aa)s,
                    %(famfr_p3p4)s, %(famfrp4_p)s, %(famfrp3_p)s, %(famfrp2_p)s, %(famfrp1_p)s, %(famfraa_p)s,
                    %(famfrp3p4p)s, %(ed_fr_p4)s, %(ed_fr_p3)s, %(ed_fr_p2)s, %(ed_fr_p1)s, %(ed_fr_aa)s,
                    %(ed_fr_p3p4)s, %(edfrp4_p)s, %(edfrp3_p)s, %(edfrp2_p)s, %(edfrp1_p)s, %(edfraa_p)s,
                    %(edfrp3p4p)s, %(im_fr_p4)s, %(im_fr_p3)s, %(im_fr_p2)s, %(im_fr_p1)s, %(im_fr_aa)s,
                    %(imfr_p3p4)s, %(imfrp4_p)s, %(imfrp3_p)s, %(imfrp2_p)s, %(imfrp1_p)s, %(imfraa_p)s,
                    %(imfrp3p4p)s, %(bbcc_fr_p4)s, %(bbcc_fr_p3)s, %(bbcc_fr_p2)s, %(bbcc_fr_p1)s, %(bbcc_fr_aa)s,
                    %(bbccfrp3p4)s, %(bbccfrp4_p)s, %(bbccfrp3_p)s, %(bbccfrp2_p)s, %(bbccfrp1_p)s, %(bbccfraa_p)s,
                    %(bbccfrp34p)s
                )
                """
                cursor.execute(insert_query, res)
                conn.commit()
        else:
            raise response.status_code
    cursor.close()
    conn.close()
except Exception as error:
    print(f"Error: {error}")